# Raster Processing Workflow
**Cara Piske, Graduate Program of Hydrologic Sciences, 2022; Advisor: Dr. Adrian Harpold**<br>
<p>This code processes rasterized LAS files. <br>
Lidar data were provided by the Airborne Snow Observatory (ASO), the National Center for Airborne Laser Mapping (NCALM), and Watershed Sciences Inc. (WSI). <br>

The goal of this project is to process snow depth to the one-meter spatial scale while maintaining conservative under-canopy estimates. Therefore, little interpolation occurs under-canopy. We follow these protocols in order to obtain a 1-m rasterized product (as opposed to the 3-m rasterized product provided by ASO on the NSIDC data portal). NCALM and WSI flights were obtained through OpenTopography.

In [15]:
# import necessary packages 
from osgeo import gdal, ogr, osr
import csv
import os
import subprocess
import sys
import pdal
import matplotlib.pyplot as plt
import numpy as np
import json
import glob
import time

In [1]:
# #make sure we're in the right working directory
# os.chdir('pat/to/dir/')
# print(os.getcwd())

# Pre-Processing

## Info

In [ ]:
# # print file info/metadata
# raster_filename = 'path/to/raster/file/filename.tif'
# gdal_info_command = ['gdalinfo',raster_filename]
# subprocess.run(gdal_info_command)

In [4]:
# # Obtain coordinates of file to use to set bounding box of merged files
# raster_filename = 'path/to/raster/file/filename.tif'
# src = gdal.Open(raster_filename) # open source file
# ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform() 
# lrx = ulx + (src.RasterXSize * xres) # lower right x
# lry = uly + (src.RasterYSize * yres) # lower right y
# src = None # close file

## Merge
Merge all files into one raster

In [11]:
gm = os.path.join('C:\\','OSGeo4W64','bin','gdal_merge.py')
gdal_calc = os.path.join('C:\\','OSGeo4W64','bin','gdal_calc.py')

**All Files in Directory**

In [5]:
# # list all files in directory that match pattern
# file_list = glob.glob('path/to/directory/*.tif')
# output_merge = 'path/to/filename/filename.tif'

In [ ]:
# # gdal_merge
# # use above coordinates or manually set coordinates for ulx, uly, lrx, lry
# cmd = ["gdal_merge.py", "-o", output_merge]#,"-ul_lr", str(ulx), str(uly), str(lrx), str(lry)]
# cmd.extend(demList)
# subprocess.call(cmd)
# # OR
# # gdal_merge
# # use above coordinates or manually set coordinates for ulx, uly, lrx, lry
# cmd = ["python",gdal_merge, "-o", output_merge]#,"-ul_lr", str(ulx), str(uly), str(lrx), str(lry)]
# cmd.extend(demList)
# subprocess.call(cmd)

## Warp
reprojection/warping utility, set the desired output coordinates (if not specified in the warping stage)

In [6]:
# # use ts to specify the x and y extents of bounding box (default 1000)
# input_tif = 'path/to/filename/filename.tif'
# output_tif = 'path/to/filename/filename.tif'
# x_ext = 1000
# y_ext = 1000
# warp_cmd = ["gdalwarp","-overwrite", input_tif, output_tif, "-ts", str(x_ext),str(y_ext)]
# subprocess.call(warp_cmd)

# Raster Calculations

In [ ]:
# calc_cmd = ['gdal_calc.py', '-A', 'input_A.tif', '-B', 'input_B.tif',
#  '--outfile', 'output_file.tif', '--calc="A*B"','--overwrite']
# subprocess.run(calc_cmd)

## Terrain Variables

In [ ]:
# gdal_terrain_cmd = ['gdaldem', 'slope', dem_input, slope_output]
# subprocess.run(gdal_terrain_cmd)

# ----------------

# Workflow Applied
We'll apply all tools above

## Merge

I ran into a path issue on the Windows machine and received an error 'OSError: [WinError 193] %1 is not a valid Win32 application'... to fix, we have to specify where the executable is - in this case, it's in our local anaconda, not on the main path 

Took this step out by merging las files in pipeline

In [20]:
pathname = '/Volumes/Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/rasterize_test_count'
output_merge = '/Volumes/Piske_lidar/SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/rasterize_test_count/ASO_SCB_20160326_rasterizeTiles_hag_count.tif'

In [21]:
merge_command = ["gdal_merge.py", "-o", output_merge]
for path in os.listdir(pathname):
    full_path = os.path.join(pathname, path)
    if os.path.isfile(full_path):
        merge_command.append(full_path)
subprocess.call(merge_command)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [ ]:
merge_command = ["python", gdal_merge, "-o", output_merge]
for path in os.listdir(pathname):
    full_path = os.path.join(pathname, path)
    if os.path.isfile(full_path):
        merge_command.append(full_path)
subprocess.call(merge_command)

## Classify Veg
Classify vegetation into tall and open classes

In [56]:
strata_neg0pt15_0pt15 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/vegStrata_neg0pt15_0pt15.tif'
strata_0pt15_2 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/vegStrata_0pt15_2.tif'
strata_2 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/vegStrata_2.tif'
strata_2_ground = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/vegStrata_2_ground.tif'
strata_2_nonground = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/veg_classes/vegStrata_2_nonground.tif'

open_strata1_2 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_open_strata1_2.tif'
ncalm_open = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_open.tif'

tall_strata1_2 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_tall_strata1_2.tif'
ncalm_tall = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tall.tif'
ncalm_tall_100 = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tall_100.tif'

veg_total = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_openORtall.tif'
veg_classified = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_vegClassified.tif'

In [64]:
# open_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15, '-B', strata_0pt15_2,
#  '--outfile', open_strata1_2, '--calc="1*logical_and(A>0,B==0)"','--overwrite']
# subprocess.run(open_cmd)

# open_cmd = ['python',gdal_calc, '-A', open_strata1_2, '-B', strata_2,
#  '--outfile', ncalm_open, '--calc="1*logical_and(A==1,B==0)"','--overwrite'] #'"--NoDataValue",'0','--overwrite']
# subprocess.run(open_cmd)

In [57]:
# tall_cmd = ['python', gdal_calc, '-A', strata_neg0pt15_0pt15, '-B', strata_0pt15_2,
#  '--outfile', tall_strata1_2, '--calc="1*logical_and(A>=0,B==0)"','--overwrite']
# subprocess.run(tall_cmd)

# tall_cmd = ['python',gdal_calc, '-A', tall_strata1_2, '-B', strata_2,
#  '--outfile', ncalm_tall, '--calc="logical_and(A==1,B>0)"','--overwrite']
# subprocess.run(tall_cmd)

# tall_cmd = ['python',gdal_calc, '-A', ncalm_tall,'--outfile', ncalm_tall_100, '--calc="A*100"','--overwrite']
# subprocess.run(tall_cmd)

In [58]:
# veg_cmd = ['python',gdal_calc, '-A', ncalm_open, '-B', ncalm_tall,
#  '--outfile', veg_total, '--calc="1*logical_or(A==1,B==1)"','--NoDataValue=0','--overwrite']
# subprocess.run(veg_cmd)

In [65]:
# veg_cmd = ['python',gdal_calc, '-A', ncalm_open, '-B', ncalm_tall_100,
#  '--outfile', veg_classified, '--calc="A+B"','--NoDataValue=0','--overwrite']
# subprocess.run(veg_cmd)

## Calculate Snow Depth

Time to bring the snow in... <br>
For each flight, we have two snow depths, filtered at 3 meters and 5 meters to use for open vs. under canopy <br>
We use the general open/short/tall/understory classifications above but refine for each flight

### Refine Vegetation Classes

In [ ]:
# snow_depth_LTET3 = 'path/to/snow_depth/less_than_equal_to_3/filename.tif'
# snow_depth_LTET5 = 'path/to/snow_depth/less_than_equal_to_5/filename.tif'
# flight_spec_short = 'path/to/flight_specific/vegetation_classes/filename.tif'
# flight_spec_open = 'path/to/flight_specific/vegetation_classes/filename.tif'
# flight_spec_understory = 'path/to/flight_specific/vegetation_classes/filename.tif'
# flight_spec_tall = 'path/to/flight_specific/vegetation_classes/filename.tif'

Refine vegetation classifications based on snow-on flights. We set a threshold of 30 cm so that if the snow-on flight is <30 cm greater than the snow-off flight, we do not keep that short/understory pixel. 

In [ ]:
# SD_open_cmd = ['python',gdal_calc, '-A', snow_depth_LTET5, '-B', ncalm_short,'-C', CHM_LT1pt5,'--NoDataValue','-9999',
#  '--outfile', flight_spec_short, '--calc="((A-C)>0.3)*B"','--overwrite']
# subprocess.run(SD_open_cmd)

# SD_open_cmd = ['python',gdal_calc, '-A', flight_spec_short, '-B', ncalm_open,'--NoDataValue','0',
#     '--outfile', flight_spec_open, '--calc="1*logical_or(A>0, B>0)"','--overwrite']
# subprocess.run(SD_open_cmd)

# SD_tall_cmd = ['python',gdal_calc, '-A', snow_depth_LTET3, '-B', ncalm_understory,'-C', CHM_LT1pt5,'--NoDataValue','-9999',
#  '--outfile', flight_spec_understory, '--calc="((A-C)>0.3)*B"','--overwrite']
# subprocess.run(SD_tall_cmd)

# SD_tall_cmd = ['python',gdal_calc, '-A', flight_spec_understory, '-B', ncalm_tall,'--NoDataValue','0',
#  '--outfile', flight_spec_tall, '--calc="1*logical_or(A>0, B>0)"','--overwrite']
# subprocess.run(SD_tall_cmd)

### Calculate Snow Depth

In [ ]:
# snow_depth_all_pts = 'path/to/filename.tif'
# snow_depth_open = 'path/to/filename.tif'
# snow_depth_tall = 'path/to/filename.tif'

In [ ]:
# SD_open_cmd = ['python',gdal_calc, '-A', snow_depth_LTET5, '-B', flight_spec_open,'--NoDataValue','-9999',
#  '--outfile', snow_depth_open, '--calc="A*B"','--overwrite']
# subprocess.run(SD_open_cmd)

# SD_tall_cmd = ['python',gdal_calc, '-A', snow_depth_LTET3, '-B', flight_spec_tall,'--NoDataValue','-9999',
#  '--outfile', snow_depth_tall, '--calc="A*B"','--overwrite']
# subprocess.run(SD_tall_cmd)

# merge_command = ["python", gdal_merge, "-o", snow_depth_all_pts, snow_depth_open, snow_depth_tall,'-a_nodata','-9999']
# subprocess.call(merge_command)

## Filter Data
No we filter the snow depths in two critical ways.<br> From Kostadinov et al, 2019 "by excluding areas
and improving the accuracy of the lidar-derived elevation datasets. Slopes greater than 30o and lake/pond water bodies as delineated by the National Hydrography Dataset (high resolution) (https://nhd.usgs.gov/) were excluded from the analysis. We used this slope threshold because the uncertainty in elevation increases dramatically above 30 degrees slope (Takahashi et al., 2005; Tinkham et al., 2012)."

**Filter out water bodies**

In [ ]:
# # Filter out all water bodies
# input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'

In [ ]:
# burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, snow_depth_open]
# subprocess.call(burn_command)
# burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, snow_depth_all_pts]
# subprocess.call(burn_command)
# burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, snow_depth_tall]
# subprocess.call(burn_command)

**Filter slopes >30 degrees**

In [ ]:
# snow_depth_refined = 'path/to/filename.tif'

In [ ]:
# sd_filter_cmd = ['python',gdal_calc, '-A', snow_depth_all_pts,'-B',slope,'--NoDataValue','0',
#  '--outfile', snow_depth_refined, '--calc="A*(B<=30)"','--overwrite']
# subprocess.run(sd_filter_cmd)

In [2]:
#open_strata = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_open_crop.tif'
#tall_strata = 'Piske_lidar/SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_tall_crop.tif'
ASO_SCB_20160326_vbc = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_tif/ASO_SCB_20160326_vbc.tif'
ASO_SCB_20160417_vbc = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/corrected_tif/ASO_SCB_20160417_vbc.tif'
ASO_SCB_20160518_vbc = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/corrected_tif/ASO_SCB_20160518_vbc.tif'
NCALM_SCB_20080210_vbc = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc.tif'

ASO_SCB_20160326_SD = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD.tif'
ASO_SCB_20160417_SD = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/snow_depth/ASO_SCB_20160417_SD.tif'
ASO_SCB_20160518_SD = 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_depth/ASO_SCB_20160518_SD.tif'
NCALM_SCB_20080210_SD = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD.tif'


In [72]:
SD_cmd = ['python',gdal_calc, '-A', veg_total, '-B', ASO_SCB_20160326_vbc,
 '--outfile', ASO_SCB_20160326_SD, '--calc="A*B"','--overwrite','--NoDataValue','-9999']
subprocess.run(SD_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_openORtall.tif', '-B', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/corrected_tif/ASO_SCB_20160326_vbc.tif', '--outfile', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160326/snow_depth/ASO_SCB_20160326_SD.tif', '--calc="A*B"', '--overwrite', '--NoDataValue', '-9999'], returncode=0)

In [68]:
SD_cmd = ['python',gdal_calc, '-A', veg_total, '-B', ASO_SCB_20160417_vbc,
 '--outfile', ASO_SCB_20160417_SD, '--calc="A*B"','--overwrite','--NoDataValue','-9999']
subprocess.run(SD_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_openORtall.tif', '-B', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/corrected_tif/ASO_SCB_20160417_vbc.tif', '--outfile', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160417/snow_depth/ASO_SCB_20160417_SD.tif', '--calc="A*B"', '--overwrite', '--NoDataValue', '-9999'], returncode=0)

In [69]:
SD_cmd = ['python',gdal_calc, '-A', veg_total, '-B', ASO_SCB_20160518_vbc,
 '--outfile', ASO_SCB_20160518_SD, '--calc="A*B"','--overwrite','--NoDataValue','-9999']
subprocess.run(SD_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_openORtall.tif', '-B', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/corrected_tif/ASO_SCB_20160518_vbc.tif', '--outfile', 'SCB/Sagehen_lidar/ASO/ASO_SCB_20160518/snow_depth/ASO_SCB_20160518_SD.tif', '--calc="A*B"', '--overwrite', '--NoDataValue', '-9999'], returncode=0)

**NCALM 2008**

In [3]:
NCALM_SCB_20080210_vbc = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc.tif'
NCALM_SCB_20080210_SD = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD.tif'
NCALM_SCB_20080210_vbc_crop = 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc_crop.tif'
input_shp = 'SCB/supporting_files/masks/NHDWaterbody_2018_EPSG26910.shp'

In [74]:
SD_cmd = ['python',gdal_calc, '-A', veg_total, '-B', NCALM_SCB_20080210_vbc,
 '--outfile', NCALM_SCB_20080210_SD, '--calc="A*B"','--overwrite','--NoDataValue','-9999']
subprocess.run(SD_cmd)

CompletedProcess(args=['python', 'C:\\Users\\cpiske\\.conda\\envs\\lidar\\Lib\\site-packages\\osgeo_utils\\gdal_calc.py', '-A', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_2014/veg_strata/NCALM_SCB_2014_openORtall.tif', '-B', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/corrected_tif/NCALM_SCB_20080210_vbc.tif', '--outfile', 'SCB/Sagehen_lidar/NCALM/NCALM_SCB_20080210/snow_depth/NCALM_SCB_20080210_SD.tif', '--calc="A*B"', '--overwrite', '--NoDataValue', '-9999'], returncode=0)

In [ ]:
gdalwarp -cutline INPUT.shp -crop_to_cutline -dstalpha INPUT.tif OUTPUT.tif


In [4]:
warp_cmd = ["gdalwarp",'-cutline',input_shp,'-crop_to_cutline', NCALM_SCB_20080210_vbc, NCALM_SCB_20080210_vbc_crop, ]
subprocess.call(warp_cmd)

1

In [ ]:
burn_command = ['gdal_rasterize', '-b', '1', '-burn', '-9999', input_shp, NCALM_SCB_20080210_vbc]
subprocess.call(burn_command)

In [ ]:
# # calculate CHM from DSM and DTM
# input_snow_depth = 'SCB/random_forest_data/ASO_sd/ASO_snow_depth_20160417_clp.tif'
# input_snow_density = 'SCB/random_forest_data/snowpalm_density/SP_density_20160417_1m.tif'
# output_swe = 'SCB/random_forest_data/calc_swe/SWE_20160417.tif'
# swe_command = ['gdal_calc.py', '-A', input_snow_depth, '-B', input_snow_density, '--calc="A*B"','--outfile', output_swe]
# subprocess.run(swe_command)